In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from processing import process_next_csv, load_graph

graph = load_graph("Madrid, Spain")
print(type(graph))

<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [10]:
from processing import process_next_csv
process_next_csv(graph, strategy="shortest")

Row 136: [420268819, 26493342, 26493335, 26493334, 11505669986, 26493355, 26493361, 26493329, 1416248973, 26487796, 1416249003, 9687726556, 26487798, 26487800, 26487804, 26487805, 26487807, 26310532, 26310526, 3272923851, 26310524, 619886896, 26341187, 21734308, 21734309, 21734297, 351959168, 351959288, 21734292, 3656019318, 390826853, 21734290, 390826846, 21953783, 21953805, 26213241, 21953989, 21954242, 1235011903, 347280274, 622103522, 26213266, 12305101539, 26213270, 318917418, 318918303, 25896238, 1179022355, 26480826, 350307051, 25896227, 3836190075, 12703330788, 793241735, 26480825, 35487795, 315522332, 305396632, 305396634, 305396629, 305396812, 305757318, 305757313, 305757309, 305396673, 265978747, 305396739, 348160678, 750119048, 4544929575, 348159085, 750118902, 348160340, 750118984, 3802745976, 335392044]
Row 137: [26494646, 616693186, 991794631, 614125247, 614125248, 1416248953, 26493333, 26493335, 26493334, 26493353, 618038027, 26493370, 26493377, 919587036, 616592945, 26

KeyboardInterrupt: 

In [39]:
import pandas as pd
import ast
df = pd.read_csv("../Data/trips_22_03_March.csv", sep =';', usecols=['geolocation_unlock', 'geolocation_lock', 'idTrip', 'idBike', 'trip_minutes'])
df = df.dropna(subset=['geolocation_unlock', 'geolocation_lock', 'idTrip'])
df = df[df['geolocation_unlock'] != df['geolocation_lock']]
df['geolocation_unlock'] = df['geolocation_unlock'].apply(ast.literal_eval)
df['geolocation_lock'] = df['geolocation_lock'].apply(ast.literal_eval)

display(df)
print(type(df['geolocation_lock'].iloc[0]))

,idTrip,idBike,trip_minutes,geolocation_unlock,geolocation_lock
1,51226187N_7837_2022-03-01T00:07:41,7837.0,7.28,"{'type': 'Point', 'coordinates': [-3.7095084, ...","{'type': 'Point', 'coordinates': [-3.7188898, ..."
3,06033371B_7292_2022-03-01T00:07:58,7292.0,7.57,"{'type': 'Point', 'coordinates': [-3.65779, 40...","{'type': 'Point', 'coordinates': [-3.6784838, ..."
5,70941410A_7925_2022-03-01T00:21:02,7925.0,19.95,"{'type': 'Point', 'coordinates': [-3.65779, 40...","{'type': 'Point', 'coordinates': [-3.717221377..."
7,X9099015P_7657_2022-03-01T00:20:48,7657.0,19.68,"{'type': 'Point', 'coordinates': [-3.7170448, ...","{'type': 'Point', 'coordinates': [-3.6615277, ..."
9,Y2625804T_5394_2022-03-01T00:12:30,5394.0,11.20,"{'type': 'Point', 'coordinates': [-3.6837876, ...","{'type': 'Point', 'coordinates': [-3.6987665, ..."
...,...,...,...,...,...
554373,Y4147544Z_1346_2022-04-01T00:15:19,1346.0,15.82,"{'type': 'Point', 'coordinates': [-3.6708959, ...","{'type': 'Point', 'coordinates': [-3.7049407, ..."
554375,03887698P_7691_2022-04-01T00:50:18,7691.0,50.70,"{'type': 'Point', 'coordinates': [-3.694475, 4...","{'type': 'Point', 'coordinates': [-3.7032777, ..."
554377,Y7644470C_589_2022-04-01T00:11:44,589.0,12.07,"{'type': 'Point', 'coordinates': [-3.6708959, ...","{'type': 'Point', 'coordinates': [-3.6751388, ..."
554379,51195963X_7250_2022-04-01T00:09:26,7250.0,9.70,"{'type': 'Point', 'coordinates': [-3.66925, 40...","{'type': 'Point', 'coordinates': [-3.646697, 4..."


<class 'dict'>


In [3]:
import networkx as nx

def inspect_graph_metadata(graph: nx.MultiDiGraph):
    report = {}

    # 1. Basic Graph Info
    report['is_directed'] = nx.is_directed(graph)
    report['number_of_nodes'] = graph.number_of_nodes()
    report['number_of_edges'] = graph.number_of_edges()
    report['is_connected'] = nx.is_weakly_connected(graph) if report['is_directed'] else nx.is_connected(graph)
    report['number_of_components'] = nx.number_weakly_connected_components(graph) if report['is_directed'] else nx.number_connected_components(graph)

    # 2. Node attribute inspection
    all_node_attrs = set()
    missing_geom_nodes = 0
    for _, node_data in graph.nodes(data=True):
        all_node_attrs.update(node_data.keys())
        if 'x' not in node_data or 'y' not in node_data:
            missing_geom_nodes += 1

    report['node_attributes'] = sorted(all_node_attrs)
    report['nodes_missing_coordinates'] = missing_geom_nodes

    # 3. Edge attribute inspection
    all_edge_attrs = set()
    missing_geom_edges = 0
    for u, v, key, edge_data in graph.edges(keys=True, data=True):
        all_edge_attrs.update(edge_data.keys())
        if 'geometry' not in edge_data:
            missing_geom_edges += 1

    report['edge_attributes'] = sorted(all_edge_attrs)
    report['edges_missing_geometry'] = missing_geom_edges

    # 4. Check for zero-length edges
    zero_length_edges = [
        (u, v, k) for u, v, k, data in graph.edges(keys=True, data=True)
        if data.get('length', 1) == 0
    ]
    report['zero_length_edges'] = len(zero_length_edges)

    # 5. Sample values from attributes (first 5)
    report['sample_node'] = next(iter(graph.nodes(data=True)), None)
    report['sample_edge'] = next(iter(graph.edges(keys=True, data=True)), None)

    # Print formatted report
    print("\n🔍 GRAPH METADATA REPORT:")
    for k, v in report.items():
        print(f"{k}: {v}")

    return report
inspect_graph_metadata(graph)


🔍 GRAPH METADATA REPORT:
is_directed: True
number_of_nodes: 67882
number_of_edges: 150386
is_connected: True
number_of_components: 1
node_attributes: ['highway', 'junction', 'railway', 'ref', 'street_count', 'x', 'y']
nodes_missing_coordinates: 0
edge_attributes: ['access', 'area', 'bridge', 'est_width', 'geometry', 'highway', 'junction', 'lanes', 'length', 'maxspeed', 'name', 'oneway', 'osmid', 'ref', 'reversed', 'service', 'tunnel', 'width']
edges_missing_geometry: 51114
zero_length_edges: 0
sample_node: (171946, {'y': 40.4212466, 'x': -3.6844432, 'highway': 'traffic_signals', 'street_count': 3})
sample_edge: (171946, 26513145, 0, {'osmid': 807334397, 'highway': 'secondary', 'lanes': '4', 'maxspeed': '50', 'name': 'Calle de Velázquez', 'oneway': True, 'reversed': False, 'length': np.float64(43.95150138023494), 'geometry': <LINESTRING (-3.684 40.421, -3.684 40.421, -3.684 40.422)>})


{'is_directed': True,
 'number_of_nodes': 67882,
 'number_of_edges': 150386,
 'is_connected': True,
 'number_of_components': 1,
 'node_attributes': ['highway',
  'junction',
  'railway',
  'ref',
  'street_count',
  'x',
  'y'],
 'nodes_missing_coordinates': 0,
 'edge_attributes': ['access',
  'area',
  'bridge',
  'est_width',
  'geometry',
  'highway',
  'junction',
  'lanes',
  'length',
  'maxspeed',
  'name',
  'oneway',
  'osmid',
  'ref',
  'reversed',
  'service',
  'tunnel',
  'width'],
 'edges_missing_geometry': 51114,
 'zero_length_edges': 0,
 'sample_node': (171946,
  {'y': 40.4212466,
   'x': -3.6844432,
   'highway': 'traffic_signals',
   'street_count': 3}),
 'sample_edge': (171946,
  26513145,
  0,
  {'osmid': 807334397,
   'highway': 'secondary',
   'lanes': '4',
   'maxspeed': '50',
   'name': 'Calle de Velázquez',
   'oneway': True,
   'reversed': False,
   'length': np.float64(43.95150138023494),
   'geometry': <LINESTRING (-3.684 40.421, -3.684 40.421, -3.684 40.42

In [4]:
import networkx as nx
from collections import defaultdict

def check_metadata_completeness(graph: nx.MultiDiGraph):
    node_field_counts = defaultdict(int)
    edge_field_counts = defaultdict(int)

    total_nodes = graph.number_of_nodes()
    total_edges = graph.number_of_edges()

    # Count node fields
    for _, data in graph.nodes(data=True):
        for field in data:
            if data[field] is not None:
                node_field_counts[field] += 1

    # Count edge fields
    for _, _, _, data in graph.edges(keys=True, data=True):
        for field in data:
            if data[field] is not None:
                edge_field_counts[field] += 1

    # Compute percentages
    print("\n🔍 NODE ATTRIBUTE COMPLETENESS")
    for field, count in sorted(node_field_counts.items()):
        percent = 100 * count / total_nodes
        print(f"  {field:20}: {percent:6.2f}%")

    print("\n🔍 EDGE ATTRIBUTE COMPLETENESS")
    for field, count in sorted(edge_field_counts.items()):
        percent = 100 * count / total_edges
        print(f"  {field:20}: {percent:6.2f}%")

    return node_field_counts, edge_field_counts
check_metadata_completeness(graph)


🔍 NODE ATTRIBUTE COMPLETENESS
  highway             :   4.94%
  junction            :   0.01%
  railway             :   0.01%
  ref                 :   0.01%
  street_count        : 100.00%
  x                   : 100.00%
  y                   : 100.00%

🔍 EDGE ATTRIBUTE COMPLETENESS
  access              :   2.23%
  area                :   0.01%
  bridge              :   0.50%
  est_width           :   0.06%
  geometry            :  66.01%
  highway             : 100.00%
  junction            :   3.28%
  lanes               :  31.63%
  length              : 100.00%
  maxspeed            :  21.33%
  name                :  53.66%
  oneway              : 100.00%
  osmid               : 100.00%
  ref                 :   0.79%
  reversed            : 100.00%
  service             :   9.04%
  tunnel              :   0.78%
  width               :   0.95%


(defaultdict(int,
             {'y': 67882,
              'x': 67882,
              'highway': 3355,
              'street_count': 67882,
              'ref': 10,
              'junction': 9,
              'railway': 4}),
 defaultdict(int,
             {'osmid': 150386,
              'highway': 150386,
              'lanes': 47561,
              'maxspeed': 32077,
              'name': 80699,
              'oneway': 150386,
              'reversed': 150386,
              'length': 150386,
              'geometry': 99272,
              'junction': 4929,
              'bridge': 749,
              'width': 1432,
              'access': 3361,
              'service': 13593,
              'ref': 1194,
              'tunnel': 1175,
              'est_width': 88,
              'area': 10}))

In [7]:
from collections import defaultdict
import osmnx as ox
import networkx as nx

def inspect_edge_metadata(graph: nx.MultiDiGraph, keys: list[str]) -> dict[str, set]:
    """
    Extracts distinct values for specified edge metadata keys from the graph.
    """
    values = defaultdict(set)

    for _, _, data in graph.edges(data=True):
        for key in keys:
            val = data.get(key)
            # For multi-value fields like highway = ['cycleway', 'residential']
            if isinstance(val, list):
                values[key].update(val)
            elif val is not None:
                values[key].add(val)

    return values

def inspect_node_metadata(graph: nx.MultiDiGraph, keys: list[str]) -> dict[str, set]:
    """
    Extracts distinct values for specified node metadata keys.
    """
    values = defaultdict(set)

    for _, data in graph.nodes(data=True):
        for key in keys:
            val = data.get(key)
            if val is not None:
                values[key].add(val)

    return values

# Your keys
edge_keys = ["bridge", "highway", "junction", "lanes", "maxspeed", "name", "oneway", "reversed", "tunnel", "width"]
node_keys = ["street_count"]

# Example usage
edge_values = inspect_edge_metadata(graph, edge_keys)
node_values = inspect_node_metadata(graph, node_keys)

# Print results
for key, vals in edge_values.items():
    print(f"\n🔍 Edge key '{key}' unique values:")
    max_print = 20
    for i, v in enumerate(sorted(vals)):
        if i >= max_print:
            print(f"  ... ({len(vals) - max_print} more values)")
            break
        print(f"  - {v}")

for key, vals in node_values.items():
    print(f"\n🔍 Node key '{key}' unique values:")
    max_print = 20
    for i, v in enumerate(sorted(vals)):
        if i >= max_print:
            print(f"  ... ({len(vals) - max_print} more values)")
            break
        print(f"  - {v}")



🔍 Edge key 'highway' unique values:
  - cycleway
  - disused
  - living_street
  - path
  - pedestrian
  - primary
  - primary_link
  - residential
  - secondary
  - secondary_link
  - service
  - tertiary
  - tertiary_link
  - track
  - trunk
  - trunk_link
  - unclassified

🔍 Edge key 'lanes' unique values:
  - 1
  - 2
  - 3
  - 4
  - 5
  - 6
  - 7
  - 8

🔍 Edge key 'maxspeed' unique values:
  - 10
  - 15
  - 20
  - 30
  - 40
  - 45
  - 5
  - 50
  - 50|30
  - 50|50|30
  - 60
  - 70
  - 80
  - 90

🔍 Edge key 'name' unique values:
  - Acceso M-45-A-3 a Gran Vía del Sureste
  - Acceso Parque de la Elipa
  - Acceso a Parking Intercambiador
  - Acceso a Urgencias
  - Acceso a la Colonia Marconi
  - Acceso galería emergencia túneles Calle 30
  - Acera Bici Serrano
  - Aeropuerto T4 Salidas
  - Anillo Verde Ciclista
  - Antigua Carretera de la Dehesa de la Villa
  - Antigua carretera M-301
  - Aparcamiento Ahorra Más
  - Aparcamiento de Edificio Aqua
  - Arroyo Valpalomero
  - Avenida Andr